# Visualisation & Simulation Examples 

You will need the to install the Python libraries [ase](https://wiki.fysik.dtu.dk/ase/) and [nglview](https://github.com/nglviewer/nglview)

e.g.: `pip install ase nglview`

Note: Its not expected that you understand in detail what is happening here. Its merely a demonstration of what can be done with a few lines of Pytho

### Visualisation of a Protein

https://www.rcsb.org/

In [ ]:
!pip install ase nglview

In [21]:
import nglview
nglview.show_pdbid("3pqr")  # load "3pqr" from RCSB PDB

NGLWidget()

###  Molecular Dynamics Simulation of Acetonitrile

In [22]:
# load the necessary functions and modules
import ase.units as units
import numpy as np
from ase.calculators.acn import ACN
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory,write, read
from ase.md import Langevin
from ase.visualize import view

# helper functions for showing output of simulation
def show_system(atoms):
    v = view(atoms, viewer="ngl")
    v.view._remote_call("setSize", target="Widget", args=["700px","700px"])
    v.view.add_ball_and_stick()
    v.view.background="black"
    return v

def print_energy(atoms):
    epot = atoms.get_potential_energy()
    ekin = atoms.get_kinetic_energy()
    temperature = atoms.get_kinetic_energy()/len(atoms)/(1.5*units.kB)
    print(f"Potential Energy: {epot:.3f} Kinetic Energy: {ekin:.3f} Temperature: {temperature:.1f}")


In [17]:
#!cat acn.xyz

In [27]:
%%time
# read coorrdinates from file
atoms = read("acn.xyz") 
tag = 'acn'

# set up some constraints, to accelerate dynamics
atoms.constraints = FixLinearTriatomic(
    triples=[(3 * i, 3 * i + 1, 3 * i + 2)
             for i in range(27)])

# define the molecular dynamics physics / rules / model
atoms.calc = ACN(rc=np.min(np.diag(atoms.cell)) / 2)
# define the simulation details, temperature, time steps, parameters, logfile name
md = Langevin(atoms, 1 * units.fs,
              temperature_K=300,
              friction=0.01,
              logfile=tag + '.log')

# define a file to save the generated structures during the dynamics
traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=10)
md.attach(print_energy,interval=10,atoms=atoms)

# start the simulation for n time steps
n_steps = 2000
md.run(n_steps)

Potential Energy: 3.760 Kinetic Energy: 21.515 Temperature: 2054.9
Potential Energy: 17.456 Kinetic Energy: 7.631 Temperature: 728.8
Potential Energy: 10.360 Kinetic Energy: 14.524 Temperature: 1387.2
Potential Energy: 1.056 Kinetic Energy: 23.544 Temperature: 2248.7
Potential Energy: 1.326 Kinetic Energy: 22.826 Temperature: 2180.1
Potential Energy: 3.954 Kinetic Energy: 19.872 Temperature: 1898.0
Potential Energy: 3.328 Kinetic Energy: 20.137 Temperature: 1923.3
Potential Energy: 3.296 Kinetic Energy: 20.091 Temperature: 1918.9
Potential Energy: 4.059 Kinetic Energy: 19.106 Temperature: 1824.8
Potential Energy: 4.248 Kinetic Energy: 18.546 Temperature: 1771.3
Potential Energy: 3.182 Kinetic Energy: 18.956 Temperature: 1810.5
Potential Energy: 2.057 Kinetic Energy: 19.740 Temperature: 1885.3
Potential Energy: 2.552 Kinetic Energy: 18.814 Temperature: 1796.9
Potential Energy: 2.527 Kinetic Energy: 18.585 Temperature: 1775.1
Potential Energy: 2.291 Kinetic Energy: 18.410 Temperature: 17

True

In [28]:
# visualization of the dynamics
traj = Trajectory("acn.traj")
show_system(traj)

### Exercise

* Find the temperature parameter. Set the temperature to 100 K and observe the result.
* The result will not be what you expect. Which parameter should be changed in addition?
* How would you change the code to continue on the previous results?